# MySQL basic


```
pip install pymysql
```

#### sha 에러

```
RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods
```

 - https://stackoverflow.com/questions/54477829/cryptography-is-required-for-sha256-password-or-caching-sha2-password

다음 패키지 설치후 재시작.

```
pip install cryptography
pip install PyMySQL[rsa]
```






## mysql 샘플 데이터베이스

강의중 실습한 userspace 스키마의 book, customer, orders 테이블 데이터를 사용.
 - bookstore_demo.sql
 

In [4]:
import pymysql

# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='user3', password = 'xogusdl1234!',
                        db='userspace', charset='utf8')

# Connection 으로부터 Cursor 생성
curs=conn.cursor()

# SQL문 실행
sql = 'select * from book'
curs.execute(sql)


13

In [5]:
 # 데이타 Fetch
rows = curs.fetchall()
rows


((1, '철락의 역사', '정론사', 7000),
 (2, '모델링 시작하기', '한비사', 13000),
 (3, 'SQL 이해', '새미디어', 22000),
 (4, '텐서플로우 시작', '새미디어', 35000),
 (5, '인공지능 개론', '정론사', 8000),
 (6, '파이썬 고급', '정론사', 6000),
 (7, '객체지향 Java', '튜링사', 20000),
 (8, 'C++ 중급', '튜링사', 13000),
 (9, 'Secure 코딩', '정보사', 7500),
 (10, 'Machine learning', 'Pearson', 13000),
 (11, '스포츠 의학', '한솔의학서적', None),
 (12, '스포츠 의학', '한솔의학서적', None),
 (14, '스포츠 의학', '한솔의학서적', None))

In [6]:
# Connection 으로부터 Cursor 생성
curs = conn.cursor()
# 고객 정보
sql = 'select * from customer'
curs.execute(sql)
# SQL문 실행

 # 데이타 Fetch
while True:
    row = curs.fetchone()
    if row == None:
        break
    print(f'{row[0]}, {row[1]}, {row[2]},{row[3]}')

1, 박지성, 영국 맨체스타,000-5000-0001
2, 김연아, 대한민국 서울,000-6000-0001
3, 장미란, 대한민국 강원도,000-7000-0001
4, 추신수, 미국 텍사스,000-8000-0001
5, 박세리, 대한민국 부산,None


In [7]:
# Connection 닫기
curs.execute("SELECT SUM(saleprice) FROM Customer, Orders\
	WHERE Customer.custid=Orders.custid\
	AND Customer.name LIKE '추신수'")
rows = curs.fetchall()
rows


((Decimal('33000'),),)

In [8]:
conn.close()

### Parameter Placeholder

동적 SQL문을 구성하기 위해서 파라미터값이 들어가는 위치에 Parameter Placeholder인 %s (MySql의 경우)를 넣고, execute() 메서드의 두번째 파라미터에 실제 파라미터값들의 Tuple을 넣어 주면 된다.

Parameter Placeholder에서는 문자열이건 숫자이건 모두 %s 를 사용하며, 문자열이라도 %s를 인용부호로 둘러싸지 않는다. 또한 한가지 주의할 점은 Placeholder는 컬럼값을 대치할 때만 사용될 수 있다. 즉 테이블이나 기타 문장에 Placeholder 를 사용할 수 없다.

In [9]:


# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='user3', password = 'xogusdl1234!',
                        db='userspace', charset='utf8')

# Connection 으로부터 Cursor 생성
curs=conn.cursor()

# SQL문 실행
sql = 'select bookid, bookname from book where bookid >= %s and bookid <= %s'



# Connection 으로부터 Dictoionary Cursor 생성

# book 도서번호가 1~10 사이 : 비교식
curs.execute(sql,(1,10))
# 데이타 Fetch

rows = curs.fetchall()
rows

((1, '철락의 역사'),
 (2, '모델링 시작하기'),
 (3, 'SQL 이해'),
 (4, '텐서플로우 시작'),
 (5, '인공지능 개론'),
 (6, '파이썬 고급'),
 (7, '객체지향 Java'),
 (8, 'C++ 중급'),
 (9, 'Secure 코딩'),
 (10, 'Machine learning'))

In [10]:

customers = ['추신수', '김연아', '박지성']
for i in customers:

    # book 도서번호가 1~10 사이 : 비교식
    sql = "SELECT Customer.name, COUNT(*) FROM Customer, Orders \
    WHERE Customer.custid=Orders.custid \
    AND Customer.name LIKE %s;"

    # 데이타 Fetch
    curs.execute(sql, i)

    # 데이타 Fetch
    rows = curs.fetchall()
    print(rows)

(('추신수', 2),)
(('김연아', 2),)
(('박지성', 3),)


## 이름

In [11]:
curs=conn.cursor()

# SQL문 실행
sql = 'select bookid, bookname from book where bookid = :Id'



# Connection 으로부터 Dictoionary Cursor 생성

# book 도서번호가 1~10 사이 : 비교식
curs.execute(sql,{'Id':1})
# 데이타 Fetch

rows = curs.fetchall()
rows

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ':Id' at line 1")

#### Dictionary 커서

cursor() 메서드는 디폴트인 Array based cursor 를 사용한다. 디폴트 Array based cursor는 Row의 결과값을 배열로 (PyMyMsql에서 정확히는 튜플) 반환한다. Row 결과를 Dictionary 형태로 리턴하도록 하는 것이 Dictionary based cursor 이고, cursor 생성시 DictCursor 옵션 사용한다.

```python
curs = conn.cursor(pymysql.cursors.DictCursor)
```


In [12]:

# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='user3', password = 'xogusdl1234!',
                        db='userspace', charset='utf8')

# Connection 으로부터 Cursor 생성
curs = conn.cursor(pymysql.cursors.DictCursor)


sql = 'select bookid, bookname from book where bookid >= %s and bookid <= %s'



# Connection 으로부터 Dictoionary Cursor 생성

# book 도서번호가 1~10 사이 : 비교식
curs.execute(sql,(1,10))
rows = curs.fetchall()
rows[:2]
# 데이타 Fetch

[{'bookid': 1, 'bookname': '철락의 역사'}, {'bookid': 2, 'bookname': '모델링 시작하기'}]

In [13]:
# 전체 행 출력:bookid, bookname


# Connection 닫기
for row in rows:
    print(row)
    print(row['bookid'], row['bookname'])


{'bookid': 1, 'bookname': '철락의 역사'}
1 철락의 역사
{'bookid': 2, 'bookname': '모델링 시작하기'}
2 모델링 시작하기
{'bookid': 3, 'bookname': 'SQL 이해'}
3 SQL 이해
{'bookid': 4, 'bookname': '텐서플로우 시작'}
4 텐서플로우 시작
{'bookid': 5, 'bookname': '인공지능 개론'}
5 인공지능 개론
{'bookid': 6, 'bookname': '파이썬 고급'}
6 파이썬 고급
{'bookid': 7, 'bookname': '객체지향 Java'}
7 객체지향 Java
{'bookid': 8, 'bookname': 'C++ 중급'}
8 C++ 중급
{'bookid': 9, 'bookname': 'Secure 코딩'}
9 Secure 코딩
{'bookid': 10, 'bookname': 'Machine learning'}
10 Machine learning


In [15]:
type(rows)

list

In [16]:
import pandas as pd
df = pd.DateFrame(rows)
df

AttributeError: module 'pandas' has no attribute 'DateFrame'

만약 Python 문자열에서 사용하는 기본 String Interpolation을 사용하면 데이타에 특수 문자가 있는 경우 SQL문 문법 오류를 발생시킬 수 있다. 예를 들어, 아래는 기본 String Interpolation을 사용한 예인데, 변수 data 안에 단일 인용부호가 있는 경우 SQL Syntax 에러를 유발시키게 된다. 또한 이러한 String Interpolation 혹은 문자열 결합(Concatenation)을 통해 동적 SQL 문을 만드는 방법은 SQL Injection 공격에 쉽게 노출되는 문제점이 있다.

```python
# 잘못된 표현
data = '서\'울'
sql = "select * from customer where category=%s and region=%s" % (1, data)
curs.execute(sql)
```

## DML

#### insert

INSERT문에 사용하는 각 컬럼값들은 직접 값을 지정(Hard code)하거나 Parameter Placeholder를 사용할 수 있다.

아래 예제는 두 개의 데이타를 INSERT 하고 Commit 하는 예이다. 여기서 한가지 주의할 점은 데이타 INSERT만 실행하고 Commit하지 않으면, 테이블의 데이타는 변경되지 않는다는 점이다.

In [17]:
# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='user3', password = 'xogusdl1234!',
                        db='userspace', charset='utf8')

# Connection 으로부터 Cursor 생성
curs = conn.cursor()

# customer에 이름, 주소, 전화번호 입력
sql = """insert into customer(name,address,phone)
         values (%s, %s, %s)"""

# ('김철수', '서울시 도봉구', '203-4943-1234')
curs.execute(sql, ('김철수', '서울시 도봉구', '203-4943-1234'))
# ('빙그레', '대구시 도봉구', '111-4943-1234')


# 커밋과 클로즈

OperationalError: (1364, "Field 'custid' doesn't have a default value")

execute() 메서드는 하나의 Row (하나의 Tuple 데이타)를 치환하여 실행하는 반면, executemany() 메소드는 복수개의 Tuple 데이타 (즉, Tuple of Tuples)를 하나의 DML 문에 적용하게 된다. 아래 예제에서 executemany()는 3개의 INSERT 문을 수행하는 것과 동일한 효과를 갖는다.

In [18]:
data = (
   ('김수철', '충청남도 계룡시', '203-4943-1234'),
   ('마동서', '강원도 원주시', '124-1234=2343')
)
sql = """insert into customer(name,address,phone)
         values (%s, %s, %s)"""
curs.executemany(sql, data)
conn.commit()

OperationalError: (1364, "Field 'custid' doesn't have a default value")

### update, delete


기존의 데이타를 수정하기 위해 UPDATE 문을 사용하고, 삭제하기 위해서 DELETE 문을 사용한다. UPDATE, DELETE 문에 사용하는 각 컬럼값들은 직접 값을 지정(Hard code)하거나 Parameter Placeholder를 사용할 수 있다.

In [19]:
# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='user3', password = 'xogusdl1234!',
                        db='userspace', charset='utf8')

try:
    # with 구문
    # INSERT: book에 (14, '스포츠 의학', '한솔의학서적') 추가
    with conn.cursor() as curs:
        sql = "delete book(bookid) values (%s)"
        curs.execute(sql, (11))
    
    # 커밋
    conn.commit()
 

    # 입력 결과 확인 with 로 select
    with conn.cursor() as curs:
        sql = "select * from book"
        curs.execute(sql)
        rs = curs.fetchall()
        for row in rs:
            print(row)

    
    
finally:
    # 클로즈
    conn.close()


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '(bookid) values (11)' at line 1")

In [20]:
# Connection 으로부터 Cursor 생성


# id=6인 사람 삭제


# 커밋과 클로즈


### try ... with

QL Connection을 열고 프로그램 중간에서 에러가 발생하면, Connection은 그대로 열려 있는 상태로 있을 수 있다. 이렇게 오픈되어 있는 Connection이 증가하면, 나중에 새로운 Connection을 오픈할 수 없게 되는데, 이를 Connection Leak 이라 부른다.

 SQL 객체들을 다룰 때 try...finally 나 with 문을 적절히 사용하여 리소스를 해제해 주는 것이 좋다.

In [24]:
# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='user3', password = 'xogusdl1234!',
                        db='userspace', charset='utf8')

try:
    # with 구문
    # INSERT: book에 (14, '스포츠 의학', '한솔의학서적') 추가
    with conn.cursor() as curs:
        sql = "insert into book(bookid, bookname, publisher) values (%s, %s, %s)"
        curs.execute(sql, (12, '스포츠 의학', '한솔의학서적'))
    
    # 커밋
    conn.commit()
 

    # 입력 결과 확인 with 로 select
    with conn.cursor() as curs:
        sql = "select * from book"
        curs.execute(sql)
        rs = curs.fetchall()
        for row in rs:
            print(row)

    
    
finally:
    # 클로즈
    conn.close()

    
    

IntegrityError: (1062, "Duplicate entry '12' for key 'book.PRIMARY'")